In [ ]:
import time

import autoshop
import pandas as pd
import pygsheets

In [ ]:
driver = autoshop.chrome.driver()

In [ ]:
autoshop.tesco.login(driver=driver)

In [ ]:
time.sleep(1)
driver.get(url="https://www.tesco.com/groceries/en-GB/orders")

In [ ]:
element = autoshop.selenium.wait_and_click(
    driver=driver,
    value="//span[text()='Make changes']",
)

In [ ]:
df_food = autoshop.google.get_all_food()

In [ ]:
list_df = []

for _, row in df_food.iterrows():
    search = row["search"]
    name = row["name"] if search is None or search == "" else search
    autoshop.logger.info(f"{name=}")

    url = autoshop.tesco.get_food_url(query=name)
    autoshop.logger.info(f"{url=}")
    driver.get(url=url)

    elements = autoshop.tesco.get_food_elements(driver=driver)

    df_temp = (
        pd.DataFrame(dict(element=elements))
        .assign(
            food=row["food"],
            name=row["name"],
            search=name,
            parent=lambda x: x["element"].apply(lambda e: e.find_element(by=autoshop.selenium.by.XPATH, value="..")),
            raw=lambda x: x["parent"].apply(lambda e: e.get_attribute("innerHTML")),
            description=lambda x: x["parent"].apply(lambda e: e.find_element(by=autoshop.selenium.by.XPATH, value=".//a[starts-with(@href, '/groceries/en-GB/products/')]//span[text() != '']").text),
            link=lambda x: x["element"].apply(lambda e: e.get_attribute("href")),
            image=lambda x: '=IMAGE("' + x["parent"].apply(autoshop.tesco.get_image_url).astype(str) + '")',
            quantity=lambda x: x["description"].apply(autoshop.tesco.get_quantity_from_description),
            amount=lambda x: pd.to_numeric(x["quantity"].apply(lambda q: q.amount), errors="coerce"),
            unit=lambda x: x["quantity"].apply(lambda q: q.unit),
            price=lambda x: pd.to_numeric(x["parent"].apply(autoshop.tesco.get_price), errors="coerce"),
            datetime=pd.Timestamp.now(),
        )
        .drop(columns=["element", "parent", "quantity"])
    )

    if df_temp.empty:
        autoshop.logger.warning(f"No data found for {name=}, {url=}")

    list_df.append(df_temp)

df = pd.concat(list_df, ignore_index=True)

In [ ]:
google_client = pygsheets.authorize(service_file='food-api-383412-1c354585502d.json')
sheet = google_client.open_by_key(key='1WVGEW5ni7xUs6o5Bs_8joM2iKZclX7hO3lTq2Wp8H-U')
worksheet = sheet.worksheet_by_title("food")

worksheet.set_dataframe(df, start=(1, 1))